# Recomendação de música

Carregando dados


In [21]:
import pandas as pd
from scipy.spatial.distance import cosine

df_musicas = pd.read_csv("musicas.csv")
df_dados = pd.read_csv("dados.csv")

Mrs = pd.read_csv("musicas_rs.csv")
Drs = pd.read_csv("dados_rs.csv")

Mminmax = pd.read_csv("musicas_minmax.csv")
Dminmax = pd.read_csv("dados_minmax.csv")

Mmaxabs = pd.read_csv("musicas_maxabs.csv")
Dmaxabs = pd.read_csv("dados_maxabs.csv")

Mstand = pd.read_csv("musicas_standart.csv")
Dstand = pd.read_csv("dados_standart.csv")



Lista de todos os artistas presentes no dataset

In [22]:
todos_artistas = df_musicas['artists'].str.split(';').explode().str.strip()
artistas_unicos = sorted(todos_artistas.unique())
pd.Series(artistas_unicos).to_csv("lista_artistas.csv", index=False)

Funções para recomendação de música

In [23]:
from scipy.spatial.distance import euclidean, cosine

def recomendar_musicas(track_id, df_dados_normalizados, df_musicas):
    idx = df_musicas.index[df_musicas['track_id'] == track_id][0]
    cluster_id = df_musicas.loc[idx, 'cluster']
    imprimir = df_musicas[df_musicas['track_id']== track_id][['artists','track_name']]
    print(f"Música: {imprimir.iloc[0]['track_name']} - {imprimir.iloc[0]['artists']}")
    musica_base = df_dados_normalizados.drop(columns='cluster').iloc[idx].values

    subset = df_dados_normalizados[df_dados_normalizados['cluster'] == cluster_id].copy()
    subset['distancia'] = subset.drop(columns='cluster').apply(lambda row: cosine(row.values, musica_base), axis=1)

    resultado_idx = subset.sort_values('distancia').index
    recomendacoes = df_musicas.loc[resultado_idx][['track_name', 'artists','track_genre']]
    recomendacoes = recomendacoes[recomendacoes.index != idx]

    return recomendacoes.head(10)


In [24]:
def recomendar_musicas_por_genero(track_id, df_dados_normalizados, df_musicas):
    # Alinhar índices
    df_dados_normalizados = df_dados_normalizados.reset_index(drop=True)
    df_musicas = df_musicas.reset_index(drop=True)

    idx = df_musicas.index[df_musicas['track_id'] == track_id][0]
    cluster_id = df_musicas.loc[idx, 'cluster']
    genero_base = df_musicas.loc[idx, 'track_genre']
    imprimir = df_musicas[df_musicas['track_id']== track_id][['artists','track_name','track_genre']]
    print(f"Música: {imprimir.iloc[0]['track_name']} - {imprimir.iloc[0]['artists']} ({imprimir.iloc[0]['track_genre']})")

    musica_base = df_dados_normalizados.drop(columns='cluster').iloc[idx].values

    # Filtra por cluster
    subset = df_dados_normalizados[df_dados_normalizados['cluster'] == cluster_id].copy()
    
    # Filtra por gênero
    subset = subset[df_musicas.loc[subset.index, 'track_genre'] == genero_base]

    # Calcula a distância
    subset['distancia'] = subset.drop(columns='cluster').apply(
        lambda row: cosine(row.values, musica_base), axis=1
    )

    resultado_idx = subset.sort_values('distancia').index
    recomendacoes = df_musicas.iloc[resultado_idx][['track_name', 'artists', 'track_genre']]
    recomendacoes = recomendacoes[recomendacoes.index != idx]

    return recomendacoes.head(10)


Buscar música por artista

In [25]:
nome_artista = "keshi"
df_musicas[df_musicas['artists'] == nome_artista][['track_id','artists', 'track_name', 'album_name']]

,track_id,artists,track_name,album_name
14167,3phI9ROeTIrkxmrywORChj,keshi,beside you,Best Road Trip Pop Tunes
14170,3SJ8r9RMlNGR4qTWBGXPrr,keshi,drunk,Halloween & Chill
14175,5IGU574p3YrKgPoSHymJHX,keshi,right here,Herbststimmung
14191,7ivYWXqrPLs66YwakDuSim,keshi,like i need u,THE REAPER
14194,1Fhb9iJPufNMZSwupsXiRe,keshi,beside you,beside you
14197,3iqlzKw1tLt6tXZyKWV0fZ,keshi,SOMEBODY,GABRIEL
14199,5SlU0Yhi51jobhEiGE4xDv,keshi,2 soon,2 soon
14237,2BuJrxYKhYky20dQqTlobO,keshi,skeletons,skeletons
14241,6D6HVKe7Qu3imn4zzJD0W9,keshi,drunk,always
14247,38umMmZQdeoOG7Zojor4g3,keshi,ANGOSTURA,GABRIEL


## Recomendação

In [26]:
id_musica = "38umMmZQdeoOG7Zojor4g3"

Robust Scale

In [27]:
recomendar_musicas(id_musica,Drs,Mrs)

Música: ANGOSTURA - keshi


,track_name,artists,track_genre
75233,You Don't Care At All,KAYDEN,sad
66631,Tear Up This Town - Orchestral Version / From ...,Keane,piano
276,Headphones,Jon Bryant,acoustic
48226,Saibo - Lofi Flip,VIBIE,indian
14382,WESTSIDE,keshi,chill
41931,Alone,Heart,hard-rock
79117,Ay Tenli Kadın,Ufuk Beydemir,singer-songwriter
59850,在青春迷失的咖啡館,Cyndi Wang,mandopop
18644,North On 21,Riley Green,country
28544,idontwannabeyouanymore,Billie Eilish,electro


In [28]:
recomendar_musicas_por_genero(id_musica,Drs,Mrs)

Música: ANGOSTURA - keshi (chill)


,track_name,artists,track_genre
14382,WESTSIDE,keshi,chill
14311,blue,keshi,chill
14258,down,Mokita,chill
14345,I Need U (Remix),yaeow;Kina,chill
15009,Grow Old with Me,Tom Odell,chill
14518,alright,keshi,chill
14197,SOMEBODY,keshi,chill
14186,17,Pink Sweat$,chill
14380,Midnight River (feat. 6LACK),Pink Sweat$;6LACK,chill
14399,By My Side,JUNNY,chill


Min Max

In [29]:
recomendar_musicas(id_musica,Dminmax,Mminmax)

Música: ANGOSTURA - keshi


,track_name,artists,track_genre
59086,不能說的秘密,Jay Chou,mandopop
55033,Zindagi Kuch Toh Bata (Reprise),Jubin Nautiyal,k-pop
42050,When I See You Smile,Bad English,hard-rock
31574,Wildfire,Seafret,folk
11215,羅生門,Juno Mak;Kay Tse,cantopop
49502,Meteor Shower,Cavetown,indie-pop
67103,"Tose Naina (From ""Mickey Virus)",Arijit Singh,pop-film
67046,Tose Naina,Arijit Singh,pop-film
84647,Power of Love,Laura Branigan,synth-pop
30171,Princess Bubblegum,ilyTOMMY,emo


In [30]:
recomendar_musicas_por_genero(id_musica,Dminmax,Mminmax)

Música: ANGOSTURA - keshi (chill)


,track_name,artists,track_genre
14501,Close to Me,Laica,chill
14905,atlas,keshi,chill
14186,17,Pink Sweat$,chill
14197,SOMEBODY,keshi,chill
14382,WESTSIDE,keshi,chill
14345,I Need U (Remix),yaeow;Kina,chill
14254,17,Pink Sweat$,chill
15009,Grow Old with Me,Tom Odell,chill
14454,pouring outside,dyl dion,chill
14427,17 (feat. Joshua and DK of SEVENTEEN),Pink Sweat$;SEVENTEEN,chill


Max Abs

In [31]:
recomendar_musicas(id_musica,Dmaxabs,Mmaxabs)

Música: ANGOSTURA - keshi


,track_name,artists,track_genre
59086,不能說的秘密,Jay Chou,mandopop
55033,Zindagi Kuch Toh Bata (Reprise),Jubin Nautiyal,k-pop
31574,Wildfire,Seafret,folk
11215,羅生門,Juno Mak;Kay Tse,cantopop
42050,When I See You Smile,Bad English,hard-rock
67046,Tose Naina,Arijit Singh,pop-film
67103,"Tose Naina (From ""Mickey Virus)",Arijit Singh,pop-film
49502,Meteor Shower,Cavetown,indie-pop
84647,Power of Love,Laura Branigan,synth-pop
59278,再見,G.E.M.,mandopop


In [32]:
recomendar_musicas_por_genero(id_musica,Dmaxabs,Mmaxabs)

Música: ANGOSTURA - keshi (chill)


,track_name,artists,track_genre
14501,Close to Me,Laica,chill
14186,17,Pink Sweat$,chill
14905,atlas,keshi,chill
14197,SOMEBODY,keshi,chill
14382,WESTSIDE,keshi,chill
14254,17,Pink Sweat$,chill
14345,I Need U (Remix),yaeow;Kina,chill
15009,Grow Old with Me,Tom Odell,chill
14427,17 (feat. Joshua and DK of SEVENTEEN),Pink Sweat$;SEVENTEEN,chill
14454,pouring outside,dyl dion,chill


Standart

In [33]:
recomendar_musicas(id_musica,Dstand,Mstand)

Música: ANGOSTURA - keshi


,track_name,artists,track_genre
79117,Ay Tenli Kadın,Ufuk Beydemir,singer-songwriter
75233,You Don't Care At All,KAYDEN,sad
276,Headphones,Jon Bryant,acoustic
78812,I Guess I Just Feel Like,John Mayer,singer-songwriter
66631,Tear Up This Town - Orchestral Version / From ...,Keane,piano
45218,Chidiya,Vilen,hip-hop
14382,WESTSIDE,keshi,chill
18644,North On 21,Riley Green,country
4868,First Love,Hikaru Utada,anime
48374,Zakir,NAALAYAK,indian


In [34]:
recomendar_musicas_por_genero(id_musica,Dstand,Mstand)

Música: ANGOSTURA - keshi (chill)


,track_name,artists,track_genre
14382,WESTSIDE,keshi,chill
14258,down,Mokita,chill
14345,I Need U (Remix),yaeow;Kina,chill
15009,Grow Old with Me,Tom Odell,chill
14518,alright,keshi,chill
14434,Silence,Weston Estate,chill
14416,I Found Her,Faime,chill
14197,SOMEBODY,keshi,chill
14186,17,Pink Sweat$,chill
14254,17,Pink Sweat$,chill


Sem normalizar

In [35]:
recomendar_musicas(id_musica,df_dados,df_musicas)

Música: ANGOSTURA - keshi


,track_name,artists,track_genre
14633,your love,brb.;Jimmy Brown,chill
14530,October Eyes,Alt Bloom,chill
30142,Tell the Reaper I'm Sorry,Lil Revive,emo
14017,Party Freeze Dance Song,The Kiboomers,children
13760,The Party Freeze Song,The Kiboomers,children
31282,The Best Is Yet to Come,Tom Walker,folk
14955,Little Do You Know?,Sivarni,chill
21894,Back to Life,HUGEL,deep-house
69291,Too Much,Dimitri Vegas & Like Mike;DVBBS;Roy Woods,progressive-house
60806,Fuckboi (feat. Conquer Divide),Electric Callboy;Conquer Divide,metalcore


In [36]:
recomendar_musicas_por_genero(id_musica,df_dados,df_musicas)

Música: ANGOSTURA - keshi (chill)


,track_name,artists,track_genre
14633,your love,brb.;Jimmy Brown,chill
14530,October Eyes,Alt Bloom,chill
14955,Little Do You Know?,Sivarni,chill
14599,The Calm Place,Zmeyev;Screen Jazzmaster;DreamBetter,chill
14992,boyfriend material,Gareth.T,chill
14536,I’ll Keep You Safe,Shiloh Dynasty;Vluestar,chill
14645,Down the Line,Beach Fossils,chill
14551,"tonight, pt. II",demxntia,chill
14283,Galaxy,Kid Travis;Eluzai,chill
14158,With You,Mokita,chill
